## Memory Types In LangChain

In this article we'll dive deep into the inner working of how chatbots can remember previous conversations in Langchain.

In previuos articles, we have gone over the basic fundamental concepts in langchain. Chatbots need to remember previous conversations for human like conversation capabilities. Without this it will be so difficult to have a conversation with a chatbot. Imagine a talking to a person who can not remember the last thing you said, this will be the case with a chatbot that can not remember the things you said earlier on.

The ability to store past information is what we refer to as **memory**. LangChain provides us with alot of utilities to add memory to our chains and chatbots. 


**A memory has two main functionalities, these are as follows:**

1. **Read:** When chain recieves an input from user, it first fetches the previous conversation data from memory to make better sense of what the follow of the conversation is.

2. **Write:** After the core logic of the chain is executed, involving input from LLM, the system then writes this to memory to keep track of what the AI's response was.

![Image](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)
[source](https://python.langchain.com/docs/modules/memory/)


## Designing Memory Into A System

There are two main things that we'll need to consider when integrating memory into a system:

1. **How messages will be stored**
2. **How messages will be retrieved**


### Storage Of Messages

In LangChain messages are stored in an in-memory database or a file(JSON) file specifically. 

### Retrieval Of Messages

When it comes to message retrieval there can be a variety of complex ways around it. A simple way would be to return all the previous messages. A more complex system will return only K last messages. A more complicated system will return a summary of the whole previous conversations or a summary of the K last messages. Other approaches will return only previous messages that match a certain token length. We'll see memory types designed for some of these approaches. These are called **memory types**.


### Memory Types

There are a couple of memory types in Langchain that we can use. Here is a list of a few of them:

1. ConversationBufferMemory
2. ConversationBufferWindowMemory
3. ConversationTokenBufferMemory
4. ConversationSummaryMemory
5. Knowledge Graph Memory
6. Entity Memory

## Loading Environment Variables

We'll use the code below to load and setup or environment variables. Make sure you have an OpenAI API key. I'll not go over how to do this as we covered this in the previous articles.

In [2]:
from dotenv import load_dotenv
import os

%load_ext dotenv
%dotenv

In [3]:
openai_api_key  = os.environ['OPANAI_API_KEY']

## ConversationBufferMemory

This is the most simplest form of memory. It simply stores all the previous chat data in a buffer and passes them to the prompt template when the logic is executed.

In [3]:
from langchain.memory import ConversationBufferMemory

In [4]:
memory = ConversationBufferMemory()

In [9]:
memory.load_memory_variables({})

{'chat_history': ''}

The above code returns the name of the variable that is retrieved from this kind of memory. Each memory type has its own variable name that is retrieved.

``{}`` the empty dictionary is a playholder that we pass in when calling the ``load_memory_variables({})``

We can also decide to name this memory variable ourselves. Here's how we can do this:

In [19]:
memory = ConversationBufferMemory(memory_key='chat_history')
memory.load_memory_variables({})

{'chat_history': ''}

#### Adding data into the memory

To add data, we can add data about what the **user input** was or what **the AI(LLM's) response** was. Here's how we do this.

In [25]:
memory.chat_memory.add_user_message("hello there!")
memory.chat_memory.add_ai_message("Hey, how can I assist you today?")

In [21]:
memory.load_memory_variables({})

{'chat_history': 'Human: hello there!\nAI: Hey, how can I assist you today?'}

In [22]:
memory.chat_memory.add_user_message("Can you tell me more aboout XYZ")
memory.chat_memory.add_ai_message("Sure, here's what you need to know about XYZ")

In [23]:
memory.load_memory_variables({})

{'chat_history': "Human: hello there!\nAI: Hey, how can I assist you today?\nHuman: Can you tell me more aboout XYZ\nAI: Sure, here's what you need to know about XYZ"}

#### Returning Messages As A String Or A List Of Messages

Messages can be returned as a string or a list of messages. It depends on where the messages are passed to. For LLMs, a simple string is best. For a ChatModel a list form is better. By default the messages are returned in form of a string. You can change this to get a list in return using ``return_message=True``

In [24]:
memory = ConversationBufferMemory(return_messages=True)

In [26]:
memory.chat_memory.add_user_message("hello there!")
memory.chat_memory.add_ai_message("Hey, how can I assist you today?")

In [31]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hello there!', additional_kwargs={}, example=False),
  AIMessage(content='Hey, how can I assist you today?', additional_kwargs={}, example=False),
  HumanMessage(content='hello there!', additional_kwargs={}, example=False),
  AIMessage(content='Hey, how can I assist you today?', additional_kwargs={}, example=False)]}

In [32]:
memory.load_memory_variables({}).get("history")

[HumanMessage(content='hello there!', additional_kwargs={}, example=False),
 AIMessage(content='Hey, how can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='hello there!', additional_kwargs={}, example=False),
 AIMessage(content='Hey, how can I assist you today?', additional_kwargs={}, example=False)]

#### Keys Saved To Memory
 
In some situations, chains (sequences of actions) can involve multiple inputs or outputs with different names. To manage this complexity, the system uses parameters called "input_key" and "output_key" associated with memory types. These parameters determine which specific input or output keys are stored in the chat message history. By default, these parameters are set to None, and if there's only one input/output key, it's automatically used. However, when there are multiple input/output keys, it's crucial to specify the particular key you want to use, ensuring clarity and control in managing the information flow.

In [35]:
from langchain import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [41]:
llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

template = """You are a friendly AI that knows all about cats

Previous conversations: {chat_history}

Human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)

# Here we need to align the memory_key to align with the one used in the prompt template string above
memory = ConversationBufferMemory(memory_key="chat_history")

chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

In [42]:
# Notice that we just pass in the `question` variables the other placeholder `chat_history` gets populated by memory
chain({"question": "Hello there, tell me the best cat food"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a friendly AI that knows all about cats

Previous conversations: 

Human question: Hello there, tell me the best cat food
Response:

> Finished chain.


{'question': 'Hello there, tell me the best cat food',
 'history': '',
 'text': " Hi there! The best cat food really depends on your cat's individual needs. Generally, you want to look for a food that is high in protein, low in carbs, and free of fillers. Also, make sure it is specifically designed for cats, as their nutritional needs are different than those of other animals."}

#### Using ChatModel

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate)

In [48]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("You are a friendly AI that knows all about cats"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# `return_messages` is set to True to get a list of messages since we are using a ChatModel
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.4)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

In [49]:
# Notice that we just pass in the `question` variables the other placeholder `chat_history` gets populated by memory
chain({"question": "Hello there, tell me the best cat food"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a friendly AI that knows all about cats
Human: Hello there, tell me the best cat food

> Finished chain.


{'question': 'Hello there, tell me the best cat food',
 'chat_history': [HumanMessage(content='Hello there, tell me the best cat food', additional_kwargs={}, example=False),
  AIMessage(content='Hello! When it comes to choosing the best cat food, there are a few factors to consider. It\'s important to look for a cat food that is nutritionally balanced and meets the specific needs of your cat. Here are some key things to consider:\n\n1. High-quality ingredients: Look for cat foods that list high-quality protein sources, such as chicken, turkey, or fish, as the main ingredient. Avoid foods that contain a lot of fillers, artificial preservatives, or by-products.\n\n2. Complete and balanced nutrition: Ensure that the cat food you choose is labeled as "complete and balanced" by the Association of American Feed Control Officials (AAFCO). This means it provides all the necessary nutrients for your cat\'s life stage, whether it\'s a kitten, adult, or senior cat.\n\n3. Life stage and specific n

## Chat Messages

The ChatMessageHistory class is a fundamental tool in memory modules, used across various scenarios. It's a simple and efficient wrapper that offers easy methods to save and retrieve both human and AI messages. If you're handling memory independently from a sequence of actions, you can directly employ this class for better management.

In [50]:
from langchain.memory import ChatMessageHistory

In [53]:
history = ChatMessageHistory()

history.add_user_message("Hi there")
history.add_ai_message("Hello, nice to hear from you")
history.messages

[HumanMessage(content='Hi there', additional_kwargs={}, example=False),
 AIMessage(content='Hello, nice to hear from you', additional_kwargs={}, example=False)]

## Types Of Memory

LangChain provides us with different types of memory classes. Each memory class has it's own usage and parameters. Let's explore these different types of memory we have.

### Conversation Buffer Memory

This is used to store messages and then extract these messages in form of variables. We have taken a look at this type of memory so far. Lets make a closer look again.

In [54]:
from langchain.memory import ConversationBufferMemory

In [56]:
memory = ConversationBufferMemory()

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})

In [57]:
memory.load_memory_variables({})

{'history': 'Human: Hi there\nAI: Hello, nice to hear from you'}

We can also get a list form as mentioned earlier. This is suitable for chat models

In [59]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi there', additional_kwargs={}, example=False),
  AIMessage(content='Hello, nice to hear from you', additional_kwargs={}, example=False)]}

#### Example Demo

In [79]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)

conversation.predict(input="Hello there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello there
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

### Conversation buffer window memory

This is similar to the `conversation buffer` memory we looked as earlier on. The major difference being that the `window` name, meaning that only a list of the previous interactions are stored. Specifically the last `k` interactions. This helps to keep the memory buffer from going  large and also reduce token cost. Since we have less text being passed to the LLM.

In [64]:
from langchain.memory import ConversationBufferWindowMemory

In [72]:
memory = ConversationBufferWindowMemory(k=2)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "Tell me about yourself, and what you enjoy doing"}, {"output": "Am an AI language model"})

memory.load_memory_variables({})

{'history': 'Human: Tell me about yourself\nAI: Am an AI language model\nHuman: Tell me about yourself, and what you enjoy doing\nAI: Am an AI language model'}

You can see the output of the above code will only keep track of the last to interactions, this means the last two inputs and thier respectice ouputs.

We can also get a list form as mentioned earlier. This is suitable for `chat models`

In [75]:
memory = ConversationBufferWindowMemory(k=2, return_messages=True)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "Tell me about yourself, and what you enjoy doing"}, {"output": "Am an AI language model"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Tell me about yourself', additional_kwargs={}, example=False),
  AIMessage(content='Am an AI language model', additional_kwargs={}, example=False),
  HumanMessage(content='Tell me about yourself, and what you enjoy doing', additional_kwargs={}, example=False),
  AIMessage(content='Am an AI language model', additional_kwargs={}, example=False)]}

#### Example Demo

In [80]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

conversation = ConversationChain(
    llm=llm,
    # keep only the last 4 interactions in buffer memory
    memory=ConversationBufferWindowMemory(k=4),
    verbose=True
)

conversation.predict(input="Hello there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello there
AI:

> Finished chain.


' Hi there! How can I help you?'

### Entity Memory

`Entity Memory` is a system that retains provided facts about certain entities within a conversation. It gathers details about these entities using a large language model (LLM) and gradually enhances its understanding of them using the same LLM over time. Essentially, it collects and stores information about specific subjects within the conversation, using a language model to both extract and expand its knowledge about those subjects. Using this type of memory is more costly as additional LLM calls will have to be made to gather details on an entity.

In [86]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

In [91]:
memory = ConversationEntityMemory(llm=llm)

_input = {"input": "Helen and Johnson are sisters and brothers from different parents. Johnson is older than Helen and was adopted by Helens' parents."}

memory.load_memory_variables(_input)

memory.save_context(
    _input,
    {"output": " They must be the coolest kids in town"}
)

In [92]:
memory.load_memory_variables({"input": 'who is Johnson'})

{'history': "Human: Helen and Johnson are sisters and brothers from different parents. Johnson is older than Helen and was adopted by Helens' parents.\nAI:  They must be the coolest kids in town",
 'entities': {'Johnson': 'Johnson is an older brother to Helen, adopted by her parents from different parents.'}}

#### Example Demo

In [95]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [96]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [97]:
conversation.predict(input="Helen and Johnson are sisters and brothers from different parents. Johnson is older than Helen and was adopted by Helens' parents.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

" That's interesting. Can you tell me more about Helen and Johnson's relationship?"

In [99]:
conversation.memory.entity_store.store

{'Helen': 'Helen is a sister to Johnson, who is older than her and was adopted by her parents.',
 'Johnson': "Johnson is an older brother to Helen, from different parents, and was adopted by Helen's parents."}

In [100]:
conversation.predict(input="They are both at the park playing football")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like fun! What position does each of them play?'

In [101]:
conversation.predict(input="They are both in the same team. With Helen playing as a defender and Johnson as a mid-fielder")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

" That's great! Do they have any other hobbies or interests they like to pursue together?"

In [102]:
conversation.predict(input="Yah, they also like to ride thier bikes in thier free time together doing races")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a lot of fun! Do they ever compete against each other?'

#### Inspecting Memory store

In [104]:
from pprint import pprint

In [105]:
pprint(conversation.memory.entity_store.store)

{'Helen': 'Helen is a sister to Johnson, who is older than her and was adopted '
          'by her parents, and is currently playing football at the park as a '
          'defender, while Johnson plays as a mid-fielder. They also enjoy '
          'riding bikes together in their free time and doing races.',
 'Johnson': 'Johnson is an older brother to Helen, from different parents, and '
            "was adopted by Helen's parents. He is currently playing football "
            'at the park, with Helen playing as a defender and Johnson as a '
            'mid-fielder, and they also enjoy riding their bikes together and '
            'doing races in their free time.',
 'pack': 'The pack is a location where Helen and Johnson play football.'}


In [106]:
conversation.predict(input="John works part-time in a company called Histon. Histon makes clothes for you children.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great job! What kind of clothes does Histon make for children?'

In [107]:
pprint(conversation.memory.entity_store.store)

{'Helen': 'Helen is a sister to Johnson, who is older than her and was adopted '
          'by her parents, and is currently playing football at the park as a '
          'defender, while Johnson plays as a mid-fielder. They also enjoy '
          'riding bikes together in their free time and doing races.',
 'Histon': 'Histon is a company that makes clothes for children.',
 'Johnson': 'Johnson is an older brother to Helen, from different parents, and '
            "was adopted by Helen's parents. He is currently playing football "
            'at the park, with Helen playing as a defender and Johnson as a '
            'mid-fielder, and they also enjoy riding their bikes together and '
            'doing races in their free time.',
 'pack': 'The pack is a location where Helen and Johnson play football.'}


### Conversation Knowledge Graph Memory


This memory variant employs a knowledge graph to reconstruct and represent memories. What's a knowledge graph?

I aksed `ChatGPT` on what a knowledge graph is and this is the response I get back, `correct response` by the way ;)

"A knowledge graph is a structured representation of information that captures relationships between different concepts, entities, or data points. It's a way to organize and store data in a format that highlights how various pieces of information are connected to each other. Knowledge graphs consist of nodes (representing entities or concepts) and edges (representing relationships between nodes).

In simpler terms, you can think of a knowledge graph as a map that shows not only individual locations (data points) but also the roads and paths (relationships) connecting those locations. This structure allows for more sophisticated and context-aware querying, reasoning, and analysis of data, making it easier to understand the connections and patterns within a dataset."

**Basic terms**

1. **Nodes ->** These are concepts, entitie or objects
2. **Edges ->** These is the information used to create a relationship between the nodes

**Semantic Enrichment**

This is the use of NLP(Natural Language Processing) to identify nodes and create relationships between the nodes.

**Example use cases**

Semantic enrichment can be used with collected user data to come up with Knowledge Graphs that can better improve you recommendations on search engines and social media platforms

For more infor on knowledge graphs, I'll recommend this [video](https://www.youtube.com/watch?v=PZBm7M0HGzw)

In [108]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI

In [110]:
llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

memory = ConversationKGMemory(llm=llm)

memory.save_context({"input": "Can you remind Jack to pick his sister from school"}, {"output": "What is the name of Jack's sister and what school is she at?"})
memory.save_context({"input": "Jack's sister is called Helen and she studeis at Histon Primary School"}, {"output": "okay"})

In [112]:
memory.load_memory_variables({"input": "Who is Helen"})

{'history': 'On Helen: Helen studies at Histon Primary School.\nOn Jack: Jack has a sister name. Jack sister attends school.'}

In [120]:
# We can also more modularly get current entities from a new message (will use previous messages as context.)
memory.get_current_entities("Jack")

['Jack', 'Helen', 'Histon Primary School']

### Getting Knowledge Triplets

Knowledge triplets, often represented in the form of a knowledge graph, are a way to encode information about relationships between entities. A knowledge triplet consists of three main components: a subject, a predicate, and an object. This format is also referred to as "Subject-Predicate-Object" (SPO). Here's what each component means:

**Subject ->** This is the entity about which the information is being stated. It's the main focus of the triplet.

**Predicate ->** The predicate describes the relationship between the subject and the object. It's like a verb that connects the subject and object and explains the nature of their connection.

**Object ->** The object is the entity to which the subject is related through the predicate. It completes the statement by specifying what the relationship means.

For example, let's say we have the triplet "Jack | loves | magoes." In this case:

1. **Subject ->** Jack
2. **Predicate ->** loves
3. **Object ->** mangoes

In [115]:
memory.get_knowledge_triplets("Jack loves mangoes")

[KnowledgeTriple(subject='Jack', predicate='loves', object_='mangoes')]

In [119]:
memory.get_current_entities("Jack")

['Jack', 'Helen', 'Histon Primary School']

#### Example Demo

In [123]:
from langchain import OpenAI
from langchain.memory import ConversationKGMemory
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain

In [128]:
llm = OpenAI(openai_api_key=openai_api_key, temperature=0.4)

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

# the order of the input_variables does not matter
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

chain_with_kg = ConversationChain(
llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm), verbose=True)

In [129]:
chain_with_kg.predict(input="Jackson is the brother of Helen. Helen gets picked up daily from school by Jackson")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Jackson is the brother of Helen. Helen gets picked up daily from school by Jackson
AI:

> Finished chain.


" Ah, so Jackson is Helen's brother. That's nice that he takes the time to pick her up from school every day. Do you have any siblings, Human?"

In [131]:
chain_with_kg.predict(input="What do you know about Helen")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Jackson: Jackson picks up Helen. Jackson is a character in story. Jackson is in love with woman named Helen. Jackson is very determined to win her heart. Jackson often puts himself in dangerous situations to help others. Jackson is a loyal friend and always has Helen's best interests in mind.
On Helen: Helen gets picked up daily from school.

Conversation:
Human: What do you know about Helen
AI:

> Finished chain.


" I know that Helen gets picked up daily from school. I also know that she is the object of Jackson's affections and that he is very determined to win her heart. He often puts himself in dangerous situations to help others and is a loyal friend who always has her best interests in mind."

In [133]:
chain_with_kg.predict(input="What is the relationship between Jackson and Helen")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Jackson: Jackson is in love with Helen. Jackson is a character in story. Jackson is in love with woman named Helen. Jackson is very determined to win her heart. Jackson often puts himself in dangerous situations to help others. Jackson is a loyal friend and always has Helen's best interests in mind. Jackson is very determined to win Helen's heart.
On Helen: Helen gets picked up daily from school.

Conversation:
Human: What is the relationship between Jackson and Helen
AI:

> Finished chain.


" Jackson and Helen are in love. Jackson is very determined to win Helen's heart and often puts himself in dangerous situations to help others. He is a loyal friend and always has Helen's best interests in mind."

### Conversation Summary Memory

The `ConversationSummaryMemory` is an advanced memory type that offers a more intricate way of storing information. This type of memory generates a condensed summary of ongoing conversations, making it beneficial for managing lengthy discussions. It continuously summarizes the conversation as it progresses and stores this summary in memory. This stored summary can then be integrated into prompts or processing chains. This memory type is particularly advantageous for lengthy discussions where using the entire message history in the prompt would consume too many tokens. In essence, it simplifies and streamlines the representation of conversation content for better efficiency and comprehension.

This type of memory is an additional cost as a call to the LLM is made to summarize the data in memory, but saves more tokens in the long run.

In [19]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain import OpenAI


llm = OpenAI(openai_api_key=openai_api_key)

memory = ConversationSummaryMemory(llm=llm)

memory.save_context({"input": "Hello there"}, {"output": "Hello, how can I assist you today?"})

In [8]:
memory.load_memory_variables({})

{'history': '\nThe human greets the AI, to which the AI responds with a question asking how it can assist.'}

In [10]:
# We can also get the history as a list of messages (this is useful if you are using this with a chat model).
memory = ConversationSummaryMemory(llm=llm, return_messages=True)
memory.save_context({"input": "Hello there"}, {"output": "Hello, how can I assist you today?"})

In [11]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human and AI greet each other, with the AI offering to assist the human.', additional_kwargs={})]}

In [14]:
# We can also utilize the predict_new_summary method directly.
messages = memory.chat_memory.messages
messages

[HumanMessage(content='Hello there', additional_kwargs={}, example=False),
 AIMessage(content='Hello, how can I assist you today?', additional_kwargs={}, example=False)]

In [17]:
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe human greets the AI, which responds by asking how it can be of assistance.'

If you possess messages beyond this class, you can effortlessly initiate the class using `ChatMessageHistory`. Upon loading, a summary will be computed.

In [20]:
history = ChatMessageHistory()
history.add_user_message("Hello there")
history.add_ai_message("Hello, how can I assist you today?")

In [21]:
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=history,
    return_messsages=True
)

In [22]:
memory.buffer

'\nThe human greets the AI, to which the AI responds by asking how it can assist them.'

You have the choice to expedite the initialization process by utilizing a pre-generated summary. This allows you to bypass the need to recreate the summary, offering a faster initialization method by directly initializing with the existing summary. This saves additional LLM call to recreate a summary hence less on token cost as well.

In [25]:
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    buffer="The human greets the AI, to which the AI responds by asking how it can assist them.",
    chat_memory=history,
    return_messsages=True
)

### Example Demo

In [26]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)
memory = ConversationSummaryMemory(llm=llm)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [27]:
chain.predict(input="Hello there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello there
AI:

> Finished chain.


' Hi there! How are you today?'

In [28]:
chain.predict(input="Pretty good")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI, to which the AI responds with a friendly greeting and inquiry about the human's wellbeing.
Human: Pretty good
AI:

> Finished chain.


" That's great to hear! How can I help you today?"

In [29]:
memory.load_memory_variables({})

{'history': "\n\nThe human greets the AI, to which the AI responds with a friendly greeting and inquiry about the human's wellbeing. The human replies that they are doing pretty good, to which the AI expresses their pleasure and asks how they can help."}

### Conversation Summary Buffer Memory

This combines two sets of memory types, the `conversation buffer window memory` and the `conversation summary`. It does two main things. Keep a certain number of previous interactions in the buffer memory as well as a summary of previous interactions that exceeded the amount of previous interaction we told it to have stored in memory. Unlike the `conversation buffer window memory` that delete previous chats that exceed the amount we instructed it to store in memory, the `conversation summary buffer memory` creates a summary of the previous interactions rather than delete them. Unlike the `conversation summary buffer memory`, instead of counting the number of interactions, this approach decides when to clear interactions based on the length of tokens used.

In [36]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=20)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "What you enjoy doing"}, {"output": "Am an AI language model, I help humans"})

memory.load_memory_variables({})

{'history': 'System: \nThe human greeted the AI and the AI responded warmly, saying it was nice to hear from them. The human then asked the AI to tell them about itself, to which the AI responded that it was an AI language model.\nHuman: What you enjoy doing\nAI: Am an AI language model, I help humans'}

In [37]:
# We can also get the history as a list of messages (this is useful if you are using this with a chat model).
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=20, return_messages=True)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "What you enjoy doing"}, {"output": "Am an AI language model, I help humans"})

memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human greeted the AI, to which the AI responded with a friendly greeting. The human then asked the AI to tell them about itself, to which the AI responded that it is an AI language model.', additional_kwargs={}),
  HumanMessage(content='What you enjoy doing', additional_kwargs={}, example=False),
  AIMessage(content='Am an AI language model, I help humans', additional_kwargs={}, example=False)]}

In [38]:
# We can also utilize the predict_new_summary method directly.
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. It enjoys helping humans, as it is an AI language model.'

#### Example Demo

In [40]:
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory


llm = OpenAI(openai_api_key=openai_api_key)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=40)

chain = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

In [41]:
chain.predict(input="Hello there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello there
AI:

> Finished chain.


' Hi there! How can I help you today?'

In [42]:
chain.predict(input="What is the largest planet in the solar system?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there
AI:  Hi there! How can I help you today?
Human: What is the largest planet in the solar system?
AI:

> Finished chain.


' The largest planet in the solar system is Jupiter. It has a diameter of 142,984 km, and it is about 11.2 times the size of Earth.'

In [43]:
chain.predict(input="Which position is it from the sun?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI, who responds with a friendly greeting and an offer to help. The human then asks what the largest planet in the solar system is.
AI:  The largest planet in the solar system is Jupiter. It has a diameter of 142,984 km, and it is about 11.2 times the size of Earth.
Human: Which position is it from the sun?
AI:

> Finished chain.


' Jupiter is the fifth planet from the sun, located between Mars and Saturn.'

In [44]:
chain.predict(input="Do you thing Mars is habitable")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI, who responds with a friendly greeting and an offer to help. The human then asks what the largest planet in the solar system is, to which the AI responds that Jupiter is the largest planet, with a diameter of 142,984 km and 11.2 times the size of Earth.
Human: Which position is it from the sun?
AI:  Jupiter is the fifth planet from the sun, located between Mars and Saturn.
Human: Do you thing Mars is habitable
AI:

> Finished chain.


" I'm sorry, I don't know."

In [45]:
memory.load_memory_variables({})

{'history': "System: \nThe human greets the AI, who responds with a friendly greeting and an offer to help. The human then asks what the largest planet in the solar system is, to which the AI responds that Jupiter is the largest planet, with a diameter of 142,984 km and 11.2 times the size of Earth, and is the fifth planet from the sun.\nAI:  Jupiter is the fifth planet from the sun, located between Mars and Saturn.\nHuman: Do you thing Mars is habitable\nAI:  I'm sorry, I don't know."}

### Conversation Token Buffer Memory

`ConversationTokenBufferMemory` maintains a memory buffer for recent interactions, and it decides when to clear interactions based on the total token length instead of the count of interactions.

In [50]:
from langchain.memory import ConversationTokenBufferMemory
from langchain import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "What you enjoy doing"}, {"output": "Am an AI language model, I help humans"})

In [51]:
memory.load_memory_variables({})

{'history': 'AI: Hello, nice to hear from you\nHuman: Tell me about yourself\nAI: Am an AI language model\nHuman: What you enjoy doing\nAI: Am an AI language model, I help humans'}

Becareful when using this memory class, because at times, it can clear the whole memory and have nothing inside of the memory in case the text it needs to store is more than the allowed `max_token_limit`. Example:

In [52]:
llm = OpenAI(openai_api_key=openai_api_key)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "What you enjoy doing"}, {"output": "Am an AI language model, I help humans"})

memory.load_memory_variables({})

{'history': ''}

In [53]:
# We can also get the history as a list of messages (this is useful if you are using this with a chat model).
llm = OpenAI(openai_api_key=openai_api_key)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40, return_messages=True)

memory.save_context({"input": "Hi there"}, {"output": "Hello, nice to hear from you"})
memory.save_context({"input": "Tell me about yourself"}, {"output": "Am an AI language model"})
memory.save_context({"input": "What you enjoy doing"}, {"output": "Am an AI language model, I help humans"})

memory.load_memory_variables({})

{'history': [AIMessage(content='Hello, nice to hear from you', additional_kwargs={}, example=False),
  HumanMessage(content='Tell me about yourself', additional_kwargs={}, example=False),
  AIMessage(content='Am an AI language model', additional_kwargs={}, example=False),
  HumanMessage(content='What you enjoy doing', additional_kwargs={}, example=False),
  AIMessage(content='Am an AI language model, I help humans', additional_kwargs={}, example=False)]}

#### Example Demo

In [56]:
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory

llm = OpenAI(openai_api_key=openai_api_key, temperature=0.5)
memory = ConversationTokenBufferMemory(llm=llm ,max_token_limit=40, return_message=True)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [57]:
chain.predict(input="Hello there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello there
AI:

> Finished chain.


' Hi there! How can I help you?'

In [58]:
chain.predict(input="What is the largest start in the Milkyway galaxy")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello there
AI:  Hi there! How can I help you?
Human: What is the largest start in the Milkyway galaxy
AI:

> Finished chain.


' The largest star in the Milky Way galaxy is the red supergiant star VY Canis Majoris. It is located in the constellation Canis Major and is estimated to have a radius of 1,420 to 2,100 times that of the Sun.'

In [59]:
memory.load_memory_variables({})

{'history': ''}

### Vector Store Retriever Memory

The `VectorStoreRetrieverMemory` stores memories in a VectorDB and retrieves the most important documents (snippets of conversation) when queried(**queries the top-K most "salient" docs every time it is called**). Unlike other memory classes, it doesn't keep track of interaction order. This approach is helpful for referring to relevant information shared earlier in the conversation, enhancing conversation context and AI responses.

In [60]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

#### Initialize your VectorStore

In [63]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings(openai_api_key=openai_api_key).embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

#### Create your the VectorStoreRetrieverMemory

In [68]:
# create retriever that will be passed to memory, K is the number of salient documents we want to retrieve each time
retriever = vectorstore.as_retriever(search_kwargs=dict(k=3))
# create actual memory
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [77]:
memory.save_context({"input": "My favourite food is Pizza"}, {"output": "Nice to know"})
memory.save_context({"input": "I like Programming"}, {"output": "Interesting, what programming language do you like most"})
memory.save_context({"input": "Python is my most loved programming language"}, {"output": "Interesting."})

print(memory.load_memory_variables({"prompt": "Do you think I really love Python"})["history"])

input: Python is my most loved programming language
output: Interesting.
input: Python is my most loved programming language
output: Interesting.
input: Python is my most loved programming language
output: Interesting.


#### Example Demo

In [78]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate


_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""

PROMPT = PromptTemplate(input_variables=["input", "history"], template=_DEFAULT_TEMPLATE)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [79]:
chain.predict(input="What is my favorite programming language?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: What is my favorite programming language?
response:  That's a great question. Do you have a favorite programming language?
input: I like Programming
output: Interesting, what programming language do you like most
input: I like Programming
output: Interesting, what programming language do you like most
Human: What is my favorite programming language?
AI:

> Finished chain.


" That's a tough question. Do you have a particular programming language that you prefer?"

In [80]:
chain.predict(input="What is my favorite programming language is Python")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Python is my most loved programming language
output: Interesting.
input: Python is my most loved programming language
output: Interesting.
input: Python is my most loved programming language
output: Interesting.
Human: What is my favorite programming language is Python
AI:

> Finished chain.


' It seems that your favorite programming language is Python.'

In [81]:
chain.predict(input="What is my favorite food?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: My favourite food is Pizza
output: Nice to know
input: What is my favorite programming language?
response:  That's a great question. Do you have a favorite programming language?
input: What is my favorite programming language?
response:  That's a tough question. Do you have a particular programming language that you prefer?
Human: What is my favorite food?
AI:

> Finished chain.


' Your favorite food is pizza.'

In [82]:
chain.predict(input="Do you think I like programming?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: I like Programming
output: Interesting, what programming language do you like most
input: I like Programming
output: Interesting, what programming language do you like most
input: I like Programming
output: Interesting, what programming language do you like most
Human: Do you think I like programming?
AI:

> Finished chain.


" That is difficult to say. I don't have enough information to make an accurate assessment. However, from your response, it seems that you do like programming. What programming language do you like the most?"